# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [6]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [7]:
import math
import functools


# Creates a list of verticies for the region of interest polygon
def getVerticies(image):
    imshape = image.shape
    height = imshape[0]
    width = imshape[1]

    topWidthPadding = 30
    bottomWidthPadding = 65
    topLeftX = width / 2 - topWidthPadding
    topRightX = width / 2 + topWidthPadding

    topY = height * .60

    return np.array([[(topLeftX, topY), #top left
        (topRightX, topY), #top right
        (width - bottomWidthPadding + 50, height), #bottom right
        (bottomWidthPadding + 50, height)]], dtype=np.int32) #bottom left
        
# Extrapolates a line segment that we'll use to represent the position
# of the lanes in the road
def extrapolate_line(x, y, slope, height):
    # find y intercept
    yIntercept = int(y - (slope * x))
    # the lowest y for the line allowed. This determines how long
    # we want the line to be
    min_y = height * .62

    # calculate the x that corresponds to the lowest y allowed
    lowX = (min_y - yIntercept) / slope

    # calculate the x the corresponds with the highest y
    highX = (height - yIntercept) / slope

    # create an list to represent the two ends of the line (x1, y1, x2, y2)
    return [lowX, min_y, highX, height]


# Given a list of lines calculate the slope to split them between the left
# and right lanes. Lines with unexpected slopes are filtered out. Next the
# lines are extrapolated and averaged with all other lines on their respective
# sides. Two lines are returned, one for the right lane, one for the left
def averageLines(lines, height):
    left = []
    right = []

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if (.5 < abs(slope) < 1):
                ext_line = extrapolate_line(x1, y1, slope, height)
                if slope < 0:
                    left.append(ext_line)
                else:
                    right.append(ext_line)

    all_lines = []
    if left:
        all_lines.append(np.mean(left, axis=0).astype(int).tolist())

    if right:
        all_lines.append(np.mean(right, axis=0).astype(int).tolist())

    return all_lines

# Applies the Grayscale transform
# This will return an image with only one color channel
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


# Applies the Canny transform
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)


# Applies a Gaussian Noise kernel
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


# Applies an image mask. Only keeps the region of the image defined
# by the polygon formed from `vertices`. The rest of the image is
# set to black.
def region_of_interest(img, vertices):
    # defining a blank mask to start with
    mask = np.zeros_like(img)

    # defining a 3 channel or 1 channel color to fill the mask with depending
    # on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    # filling pixels inside the polygon defined by "vertices" with the
    # fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    # returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


# This function draws `lines` with `color` and `thickness`.
# Lines are drawn on the image inplace (mutates the image).
def draw_lines(img, lines, color=[255, 0, 0], thickness=15):
    for x1, y1, x2, y2 in lines:
        cv2.line(img, (x1, y1), (x2, y2), color, thickness)


# 'img' should be the output of a Canny transform. Draws hough lines on
# this image.
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold,
                            np.array([]), minLineLength=min_line_len,
                            maxLineGap=max_line_gap)
    lines = averageLines(lines, img.shape[0])
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)

    return line_img


# `img` is the output of the hough_lines(), An image with lines drawn on it.
# Should be a blank image (all black) with lines drawn on it.
# `initial_img` should be the image before any processing.
# NOTE: initial_img and img must be the same shape!
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [8]:
import os
os.listdir("test_images/")

['solidWhiteCurve.jpg',
 'solidWhiteRight.jpg',
 'solidYellowCurve.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg']

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [9]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [10]:
# Processes the image and returns the original image with lane lines drawn on it
def process_image(image):
    result = grayscale(image)
    result = gaussian_blur(result, 5)
    result = canny(result, 50, 100)
    result = region_of_interest(result, getVerticies(result))
    result = hough_lines(result, 2, np.pi/180, 15, 30, 20)
    return weighted_img(result, image)

Let's try the one with the solid white lane on the right first ...

In [11]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:05<00:00, 42.20it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 2.74 s, sys: 470 ms, total: 3.21 s
Wall time: 6.12 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Overall, my algorithm is pretty basic and isn't really ready for production use. But hey, not bad for project 1!
First, the algorithm produces fairly jittery lines. We could better by averaging the positions of lines from a couple previous frames. 
The algorithm will fail if the road is missing a couple lane lines on either direction or in snowy conditions where you can't see the lane lines at all! 
Also, as seen in the challenge video, the algorithm freaks out when the road color changes from black to white. 


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))